In [ ]:
import pickle
import networkx as nx
import pandas as pd
import herbSyner_Finder
from herbSyner_Finder import Herb_Comb_Community
from herbSyner_Finder import Detection_key_Module_multiple
from herbSyner_Finder import herb_synergy_landscape


# 1.  calculate network distance

In [ ]:
# set save file path 

save_path = "result"

HIf = 'result/herb_ingredient_pairs_pd.csv' # herb_ingre_pairs
HHf = 'result/herb_herb_dis_pd.csv' # herb_herb_dis
IIf = 'result/ingre_ingre_dis_pd.csv' # ingre_ingre_dis
DHf = 'result/herb_disease_pd.csv' # herb_disease_pd
DIf = 'result/herb_disease_ingre_pd.csv' # herb_disease_ingre
disease_name = "Cough Variant Asthma"
disease_list = ['Cough Variant Asthma']


## 1.calculate network distance 

In [ ]:
%%time

# use herb pair 'MA HUANG','ZI SU YE' as example

combination_herb_list=['MA HUANG','ZI SU YE']

herb_list, herb_herb_dis, ingre_ingre_dis, herb_ingre_pairs = herb_synergy_landscape.cal_combination_distance(herb_info,combination_herb_list)

# here， we use random time =  1 as example to save time
herb_disease_pd, herb_disease_ingre_pd = herb_synergy_landscape.cal_combination_disease(herb_list, herb_distance_obj, disease_list, random_time=1)
    

In [ ]:
# save distance data from example as files
herb_herb_dis.to_csv('result/herb_herb_distance_example.csv', index=None) 
ingre_ingre_dis.to_csv('result/ingre_ingre_distance_example.csv', index=None)
herb_ingre_pairs.to_csv('result/herb_ingredient_pairs_example.csv', index=None)
herb_disease_pd.to_csv('result/herb_disease_pd_example.csv', index=None)
herb_disease_ingre_pd.to_csv('result/herb_disease_ingre_pd_example.csv', index=None)

# 2. calculate the commnunities for the complex landscape between herbs

In [ ]:
# reload data generated by step 1
HH = pd.read_csv(HHf).drop_duplicates()
II = pd.read_csv(IIf).drop_duplicates()
DH = pd.read_csv(DHf).drop_duplicates()
DI = pd.read_csv(DIf).drop_duplicates()

In [ ]:
ADMETf = 'source_data/ingredient_ADMET_Properties.xlsx'
ADMET = pd.read_excel(ADMETf).drop_duplicates()

In [ ]:
%%time 

herb_commuties_case_obj = Herb_Comb_Community(HH, II, DH, DI, disease_name)
G, communities, DHI_result, subset_data_I, subset_data_H, node_detect_df = herb_commuties_case_obj.get_herb_communities(ADMET_filter = True)

# 3. Further detect the key module with main synergistic effects

In [ ]:
# prepare necessary files
comb_data_ingre_list = [subset_data_I]
comb_name_ingre_list = ['SH']
comb_data_herb_list = [subset_data_H]
comb_name_herb_list = ['SH']
comb_data_merged_list = [herb_commuties_case_obj.sum_file_all]
comb_name_merged_list = ['SH']
mocule_dataset_list = [comb_data_ingre_list, 
                       comb_name_ingre_list,
                       comb_data_herb_list,
                        comb_name_herb_list,
                        comb_data_merged_list,
                        comb_name_merged_list]

In [ ]:
%%time
 
# Detection_key_Module_multiple
module_multiple = Detection_key_Module_multiple(*mocule_dataset_list)
module_multiple.prepare_downstream_data(save_path)